```
QUERYNAME = CholesterolMSMS;
DEFINE PR = 'C[41..49] H[30..200] O[2] N[1]' WITH DBR = (4.5,10.5), CHG = +1;
DEFINE FR ='C[27] H[45]' WITH DBR = (1,8), CHG = +1;

IDENTIFY

	PR IN MS1+ AND
	FR in MS2+

SUCHTHAT
	(avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
	PR.chemsc[C]/PR.chemsc[db] >= 4.6

REPORT 

	SPECIE = "CE %d:%d" % "((PR.chemsc)[C] - 27, (PR.chemsc)[db] - 4.5)";  
	CLASS = "CE";
	MASS = PR.mass;
	ISOBARIC = PR.isobaric;
	CHEMSC = PR.chemsc;
	ERRppm = "%2.2f" % "(PR.errppm)";
	FRERRppm = "%2.2f" % "(FR.errppm)";
	INT = FR.intensity;
	QUALA = PR.intensity;
```

In [1]:
from targets import MFQL_util
from MS_reader import SpectraUtil

In [2]:
QUERYNAME = 'CholesterolMSMS'

In [3]:
elements = {'C':(41,49),'H':(30,200),'O':(2,2),'N':(1,1),'P':(1,1) }      
PR = MFQL_util(elements)
PR.set_dbr(4.5,10.5)

In [4]:
elements = {'C':(27,27),'H':(45,45) }      
FR = MFQL_util(elements)
FR.set_dbr(1,8)

In [5]:
FR._df.chem

0    C27 H45
Name: chem, dtype: object

In [6]:
filename = '..\\test_resources\\small_test\\190321_Serum_Lipidextract_368723_01.mzML'
spectraUtil = SpectraUtil.fromFile(filename, test_sample=True)

In [7]:
MS1 = spectraUtil
MS2 = spectraUtil.get_reset_copy()
MS1.set_mode()
MS1.set_ms_level()
MS2.set_mode()
MS2.set_ms_level(2)


a copy of the original with nothing set... sorry no undo
set mode to positive : True, (false means negative) 
set ms level to  : 1
set mode to positive : True, (false means negative) 
set ms level to  : 2


In [8]:
MS1_match = MS1.get_nearest(PR._df)
MS2_match = MS2.get_nearest(FR._df)

find the nearest Peaks to the target_peaks with a tolerance of 0.01
find the nearest Peaks to the target_peaks with a tolerance of 0.01


## get only the nearest peak in each scan

In [9]:
MS1_match.shape

(302, 12)

In [10]:
MS1_match = MFQL_util.keepMinErrorPeaks(MS1_match)
MS2_match = MFQL_util.keepMinErrorPeaks(MS2_match)

C:\Users\mirandaa\Anaconda3\envs\lx128_p3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
MS1_match.shape

(296, 13)

In [12]:
all_match = MFQL_util.makeAllCombo(MS1_match, MS2_match)

In [13]:
all_match.columns

Index(['PR_id', 'PR_m', 'PR_i', 'PR_index', 'PR_C', 'PR_H', 'PR_O', 'PR_N',
       'PR_P', 'PR_dbr', 'PR_chem', 'PR_target', 'PR_err', 'FR_id', 'FR_m',
       'FR_i', 'FR_index', 'FR_C', 'FR_H', 'FR_dbr', 'FR_chem', 'FR_target',
       'FR_err'],
      dtype='object')

In [18]:
# # (avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
# 	PR.chemsc[C]/PR.chemsc[db] >= 4.6

ST = all_match.query('FR_i/PR_i >= 0.1 and FR_i/PR_i <= 10 and PR_C/PR_dbr >= 4.6')

# make it a summary like the current lx, 
# one entry one line


In [19]:
pr_df, fr_df = MFQL_util.devideAddCombo(ST)

c:\Users\mirandaa\Desktop\lx128_p3\lipidxplorer\lx2\targets.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df.sort_values(['PR_err', 'FR_err'], inplace = True)


In [20]:
pr_df[pr_df.PR_chem== 'C41 H76 O2 N1 P1']

,PR_id,PR_m,PR_i,PR_index,PR_C,PR_H,PR_O,PR_N,PR_P,PR_dbr,PR_chem,PR_target,PR_err
784,controllerType=0 controllerNumber=1 scan=8,645.561218,84917.601562,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.000122
757,controllerType=0 controllerNumber=1 scan=17,645.559509,255222.156250,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.001831
730,controllerType=0 controllerNumber=1 scan=21,645.559509,153122.875000,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.001831
703,controllerType=0 controllerNumber=1 scan=22,645.559509,154247.328125,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.001831
676,controllerType=0 controllerNumber=1 scan=14,645.559387,237835.984375,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.001953
650,controllerType=0 controllerNumber=1 scan=20,645.559326,468750.468750,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.002014
622,controllerType=0 controllerNumber=1 scan=13,645.559265,627203.062500,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.002075
595,controllerType=0 controllerNumber=1 scan=11,645.559204,516790.125000,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.002136
569,controllerType=0 controllerNumber=1 scan=24,645.559143,408192.218750,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.002197
541,controllerType=0 controllerNumber=1 scan=23,645.559082,777978.062500,46.0,41.0,76.0,2.0,1.0,1.0,4.5,C41 H76 O2 N1 P1,645.56134,0.002258


In [21]:
sum_df = MFQL_util.summaryDF(pr_df)
sum_df

,PR_chem,PR_m_mean,PR_m_std,PR_ppm_mean,PR_i_mean,PR_i_std,PR_count
34,C49 H85 O2 N1 P1,750.640686,0.000642,11.925718,95714.500000,40528.929688,3
3,C42 H69 O2 N1 P1,650.498413,NaN,12.572833,134658.078125,NaN,1
35,C49 H88 O2 N1 P1,753.664856,NaN,12.714725,55418.605469,NaN,1
26,C47 H84 O2 N1 P1,725.633789,0.000122,13.584416,150926.343750,16780.466797,4
